<a href="https://colab.research.google.com/github/Ishan1923/Skin-Cancer-Detection-using-DL-techniques/blob/main/SkinCancerDetectionUsingVGG16Arch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
nodoubttome_skin_cancer9_classesisic_path = kagglehub.dataset_download('nodoubttome/skin-cancer9-classesisic')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import glob
from keras import layers
from tensorflow import data as tf_data

In [ ]:
# !pip install tensorflow-gpu
!nvidia-smi

In [ ]:
print("GPU available: ", tf.config.list_physical_devices('GPU'))

In [ ]:
import kagglehub

# Download the dataset
dataset_path = kagglehub.dataset_download("nodoubttome/skin-cancer9-classesisic")

print("Path to dataset files:", dataset_path)

In [ ]:
print("current directory contents: \n", os.listdir(dataset_path))

In [ ]:
ISIC_folder = os.path.join(dataset_path, os.listdir(dataset_path)[0])
print(ISIC_folder)

In [ ]:
print(os.listdir(ISIC_folder))

In [ ]:
train_image_folder = os.path.join(ISIC_folder, os.listdir(ISIC_folder)[0])
print(train_image_folder)

In [ ]:
os.listdir(train_image_folder)

In [ ]:
test_image_folder = os.path.join(ISIC_folder, os.listdir(ISIC_folder)[0])
print(test_image_folder)

In [ ]:
os.listdir(train_image_folder)

In [ ]:
os.listdir(test_image_folder)

In [ ]:
train_images_address = []
train_images_labels = []
test_images_address = []
test_images_labels = []

count = 0
for folder in os.listdir(train_image_folder):
    folder_path = os.path.join(train_image_folder, folder)
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        train_images_address.append(file_path)
        train_images_labels.append(count)
    count = count + 1

count = 0
for folder in os.listdir(test_image_folder):
    folder_path = os.path.join(test_image_folder, folder)
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        test_images_address.append(file_path)
        test_images_labels.append(count)
    count = count + 1

print("total classes in training folder: ", len(train_images_address))
print("total classes in test folder: ", len(test_images_address))
print("total training images: ",len(train_images_labels))
print("total test images", len(test_images_labels))



In [ ]:
class TFPreprocessingPipiline:
  def __init__(self, img_size = (224,224), batch_size = 32):
    self.img_size = img_size
    self.batch_size = batch_size

  def preprocess_image(self, image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels = 3)
    image = tf.reshape(image, [tf.shape(image)[0], tf.shape(image)[1], 3])
    image = tf.image.resize(image, self.img_size)
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

  def preprocess_with_augmentation(self, image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta = 0.2)
    image = tf.image.random_contrast(image, lower = 0.8, upper = 1.2)
    image = tf.image.random_contrast(image, lower = 0.8, upper = 1.2)


    k = tf.random.uniform([], 0, 4, tf.int32)
    image = tf.image.rot90(image, k)

    image = tf.clip_by_value(image, 0, 1)

    return image, label

  def create_dataset_from_directory(self, data_dir, validation_split = 0.2, subset = 'training', augment = True):
    dataset = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split = validation_split,
        subset = subset,
        seed = 42,
        image_size = self.img_size,
        batch_size = self.batch_size,
        label_mode = 'categorical'
    )

    if augment and subset == 'training':
      dataset = dataset.map(self.apply_augmentation_to_batch, num_paraller_calls = tf.data.AUTOTUNE)

    dataset = dataset.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y), num_parallel_calls = tf.data.AUTOTUNE)
    dataset = dataset.cache()
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset

  def apply_augmentation_to_batch(self, images, labels):
    augmented_images = tf.map_fn(
        lambda img: self.augment_single_image(img),
        images,
        parallel_iterations = 10,
        dtype = tf.float32
    )
    return augmented_images, labels

  def augment_single_images(self, images):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, lower=0.9, upper=1.1)
    return image

  def create_dataset_from_paths_and_labels(self, image_paths, labels, augment = True):
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))

    dataset = dataset.map(self.preprocess_image, num_parallel_calls = tf.data.AUTOTUNE)

    if augment :
      dataset = dataset.map(self.preprocess_with_augmentation, num_parallel_calls = tf.data.AUTOTUNE)
    else:
      dataset = dataset.map(self.preprocess_image, num_parallel_calls = tf.data.AUTOTUNE)

    dataset = dataset.shuffle(buffer_size = 1000)
    dataset = dataset.batch(self.batch_size)
    dataset = dataset.cache()
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset

  def create_dataset_for_prediction(self, image_paths):
    dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    dataset = dataset.map(
        lambda path : self.preprocess_image_only(path),
        num_parallel_calls = tf.data.AUTOTUNE
        )

    dataset = dataset.batch(self.batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset

  def preprocess_image_only(self, image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels = 3)
    image = tf.reshape(image, [tf.shape(image)[0], tf.shape(image)[1], 3])
    image = tf.image.resize(image, self.img_size)
    image = tf.cast(image, tf.float32) / 255.0
    return image

  def visualize_batch(self, dataset, num_images = 9):
    import matplotlib.pyplot as plt

    for images in dataset.take(1):
      fig, ax = plt.subplots(3,3, figsize = (10,10))
      ax = ax.ravel()

      for i in range(min(num_images, len(images))):
        img = images[i].numpy()
        ax[i].imshow(img)
        ax[i].axis('off')

      plt.tight_layout()
      plt.show()
      break



In [ ]:
# preprocessing_pipeline = PreprocessingTrainingData()
# training_dataset = preprocessing_pipeline.create_dataset(train_images_address, train_images_labels)
# test_dataset = preprocessing_pipeline.create_dataset_for_prediction(test_images_address)

In [ ]:
# training_dataset.take(1)

In [ ]:
# preprocessing_pipeline.visualize_batch(training_dataset)

In [ ]:
print(len(train_images_address), len(train_images_labels))

In [ ]:
print(len(test_images_address), len(test_images_labels))

In [ ]:
pipeline = TFPreprocessingPipiline(img_size=(224, 224), batch_size = 32)

train_dataset = pipeline.create_dataset_from_paths_and_labels(train_images_address, train_images_labels)

print(f"dataset: {train_dataset}")

test_dataset = pipeline.create_dataset_from_paths_and_labels(test_images_address, test_images_labels)
t_dataset = pipeline.create_dataset_for_prediction(test_images_address)
print(f"dataset: {test_dataset}")

train_dataset.take(1)

In [ ]:
pipeline.visualize_batch(t_dataset)

### Building the VGG16 Architecture Model

In [ ]:
def model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    # Rescaling layer
    x = keras.layers.Rescaling(scale=1.0 / 255.0)(inputs)

    # First conv block
    x = keras.layers.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        padding="same",
        strides=1,
        activation="relu"
    )(x)
    x = keras.layers.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        padding="same",
        strides=1,
        activation="relu"
    )(x)
    x = keras.layers.MaxPooling2D(
        pool_size=(2, 2),
        strides=2
    )(x)

    # Second conv block
    x = keras.layers.Conv2D(
        filters=128,
        kernel_size=(3, 3),
        padding="same",
        strides=1,
        activation="relu"
    )(x)
    x = keras.layers.Conv2D(
        filters=128,
        kernel_size=(3, 3),
        padding="same",
        strides=1,
        activation="relu"
    )(x)
    x = keras.layers.MaxPooling2D(
        pool_size=(2, 2),
        strides=2
    )(x)

    # Third conv block
    x = keras.layers.Conv2D(
        filters=256,
        kernel_size=(3, 3),
        padding="same",
        strides=1,
        activation="relu"
    )(x)
    x = keras.layers.Conv2D(
        filters=256,
        kernel_size=(3, 3),
        padding="same",
        strides=1,
        activation="relu"
    )(x)
    x = keras.layers.Conv2D(
        filters=256,
        kernel_size=(3, 3),
        padding="same",
        strides=1,
        activation="relu"
    )(x)
    x = keras.layers.MaxPooling2D(
        pool_size=(2, 2),
        strides=2
    )(x)

    # Fourth conv block
    x = keras.layers.Conv2D(
        filters=512,
        kernel_size=(3, 3),
        padding="same",
        strides=1,
        activation="relu"
    )(x)
    x = keras.layers.Conv2D(
        filters=512,
        kernel_size=(3, 3),
        padding="same",
        strides=1,
        activation="relu"
    )(x)
    x = keras.layers.Conv2D(
        filters=512,
        kernel_size=(3, 3),
        padding="same",
        strides=1,
        activation="relu"
    )(x)
    x = keras.layers.MaxPooling2D(
        pool_size=(2, 2),
        strides=2
    )(x)

    # Flatten before dense layers
    x = keras.layers.Flatten()(x)

    # Fully connected layers
    x = keras.layers.Dense(
        units=4096,
        activation="relu"
    )(x)
    x = keras.layers.Dropout(0.5)(x)

    x = keras.layers.Dense(
        units=4096,
        activation="relu"
    )(x)
    x = keras.layers.Dropout(0.5)(x)

    # Output layer
    outputs = keras.layers.Dense(
        units=num_classes,
        activation="softmax"
    )(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model



In [ ]:
print(len(os.listdir(train_image_folder)) )

In [ ]:
model = model(input_shape=(224, 224, 3), num_classes=len(os.listdir(train_image_folder)))

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model, show_shapes = True)

In [ ]:
callbacks = [
        keras.callbacks.EarlyStopping(
            patience=5,
            restore_best_weights=True
        ),
        keras.callbacks.ReduceLROnPlateau(
            factor=0.1,
            patience=3
        )
    ]

In [ ]:
train_dataset

In [ ]:
test_dataset

In [ ]:
# history = model.fit(
#     train_dataset,
#     batch_size=1,
#     epochs=1,
#     validation_data=test_dataset,
#     callbacks=callbacks,
#     verbose=1
# )

In [ ]:
# test_loss, test_accuracy = model.evaluate(test_dataset, verbose=0)
# print(f"Test Accuracy: {test_accuracy:.4f}")

In [ ]:
from keras.applications import VGG16 as vgg16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model as models

base_model = vgg16(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=len(os.listdir(train_image_folder)),
    classifier_activation="softmax",
    name="vgg16",
)
base_model.trainable = False

x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(256, activation = "relu")(x)
x = layers.Dropout(0.5)(x)

num_classes = len(os.listdir(train_image_folder))
output = layers.Dense(num_classes, activation="softmax")(x)

model = models(inputs = base_model.input, outputs = output)
model.summary()

In [ ]:
keras.utils.plot_model(model, show_shapes = True)

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)
callbacks = [
        keras.callbacks.EarlyStopping(
            patience=5,
            restore_best_weights=True
        ),
        keras.callbacks.ReduceLROnPlateau(
            factor=0.1,
            patience=3
        )
    ]

In [ ]:
train_dataset

In [ ]:
model.fit(
    x=train_dataset,
    batch_size=32,
    epochs=50,
    verbose=1,
    callbacks=callbacks,
    validation_data = test_dataset,
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
save_dir = "/content/drive/MyDrive/models/vgg"
os.makedirs(save_dir, exist_ok=True)

model.save(os.path.join(save_dir, "my_model.keras"))
model.save("/content/drive/MyDrive/my_model.keras")

In [ ]:
model

In [ ]:
import matplotlib.pyplot as plt

plt.subplot(1,2,1)
plt.plot(model.history.history['accuracy'], label = 'train accucracy')
plt.plot(model.history.history['val_accuracy'], label = 'value accucracy')
plt.title("Epochs vs Accuracy")
plt.xlabel("epochs")
plt.ylabel("accuarcy(sparse categorical loss)")
plt.grid(1)
plt.legend()

plt.subplot(1,2,2)
plt.plot(model.history.history['loss'], label = 'train loss')
plt.plot(model.history.history['val_loss'], label = 'value loss')
plt.title("Epochs vs Loss")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.legend()
plt.grid(1)

plt.show()

In [ ]:
loss, accuracy = model.evaluate(test_dataset)
print(f"loss : {loss}, accuracy : {accuracy}")